In [2]:
import numpy as np 
# import pandas as pd
import time
import json
import logging 
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Chrome


In [29]:
import re

In [30]:
from utils_sgpools.sgpools_regex_utils import regex_bettype_filter

In [31]:
asianhandicap_text="Tue, 11 Jun 2024\n6.14pm\n5828\nJapan vs Syria\n(+12)\n-Asian Handicap\n(Account Only)\n-\n1.88\nJapan -2.25 \n-\n1.93\nSyria +2.25 \n7.00pm\n5841\nKorea Republic vs China PR\n(+12)\n-Asian Handicap\n(Account Only)\n-\n1.88\nKorea Republic -1.75 \n-\n1.93\nChina PR +1.75 \n9.00pm\n5873\nMalaysia vs Chinese Taipei\n(+16)\n-Asian Handicap\n(Account Only)\n-\n1.85\nMalaysia -2.75 \n-\n1.87\nChinese Taipei +2.75     "


In [99]:
betdetails=re.findall(r'\n(\d{4})\n.*?\n.*?\n.*?\n.*?\n.*?\n.*?([\d|\.]*)\n.*?([\+|\-][\d|\.]*).*?\n.*?\n.*?([\d|\.]*)\n.*?([\+|\-][\d|\.]*)',asianhandicap_text,re.MULTILINE)
print(betdetails)


[('5828', '1.88', '-2.25', '1.93', '+2.25'), ('5841', '8', '-1.75', '1.93', '+1.75'), ('5873', '1.85', '-2.75', '1.87', '+2.75')]


In [14]:


def event_filter (event_string:str):

    date_pattern = r'^(\w{3}, \d{2} \w{3} \d{4})$'
    time_pattern = r'^(\d{1,2}.\d{2}(am|pm))$'
    match_pattern = r'^(\d+)$'
    teams_pattern = r'^([\w\s]+) vs ([\w\s]+)$'
    handicap_pattern = r'^\(([-+]?\d+)\)$'
    score_pattern = r'^(\d+)/(\d+)\s([\w\s]+)$'
    odds_pattern = r'^(\d{2})\n(\d+\.\d+)\n([\w\s]+ [-+]\d+\.\d+)\s\n(\d{2})\n(\d+\.\d+)\n([\w\s]+ [-+]\d+\.\d+)$'

    # Find matches for each pattern
    date_match = re.search(date_pattern, event_string, re.MULTILINE)
    time_match = re.search(time_pattern, event_string, re.MULTILINE)
    match_id_match = re.search(match_pattern, event_string, re.MULTILINE)
    teams_match = re.search(teams_pattern, event_string, re.MULTILINE)
    handicap_match = re.search(handicap_pattern, event_string, re.MULTILINE)
    score_match = re.search(score_pattern, event_string, re.MULTILINE)
    odds_match = re.search(odds_pattern, event_string, re.MULTILINE)

    # Extract matched components
    date_of = date_match.group(1) if date_match else None
    time_of = time_match.group(1) if time_match else None
    match_id = match_id_match.group(1) if match_id_match else None
    team1, team2 = teams_match.groups(1) if teams_match else (None, None)
    handicap = handicap_match.group(1) if handicap_match else None
    # home_score, away_score, goal = score_match.groups() if score_match else (None, None, None)
    # home_odds, away_odds = odds_match.group(2), odds_match.group(5) if odds_match else (None, None)

    # Output the extracted components
    # print("Date:", date_of)
    # print("Time:", time_of)
    # print("Match ID:", match_id)
    # print("Teams:", team1, "vs", team2)
    # print("Handicap:", handicap)
    # print("Score:", home_score, "/", away_score, goal)
    # print("Home Odds:", home_odds)
    # print("Away Odds:", away_odds)

    return date_of,time_of,match_id,team1,team2, handicap


In [19]:

target_url = "https://online.singaporepools.com/en/sports/category/1/football"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get(target_url)

#writes pure bet values
with open("sg_pools2.json", "w") as json_file:
    json_file.write("")

#writes only bet types
with open("sg_pools.json", "w") as json_file:
    json_file.write("")

# WebDriverWait(driver,20)
time.sleep(3)

# Get all bet types possible
bet_type_specific = driver.find_element(By.XPATH, "//select[@class='form-control event-list__filter__market-sort']")
bet_type_list=bet_type_specific.text.split('\n')
# bet_type_list=bet_type_specific.json()
all_bet_list = {'bet_list':bet_type_list}
with open("sg_pools.json", "a") as json_file:
    json_file.write(json.dumps(all_bet_list))

testing_data = {"1/2 Goal": ["Sun, 21 Jan 2024", "12.30am", "1028", "Bordeaux vs Nice", "(+17)", "021/2 Goal", "01", "1.20", "Bordeaux +1.5 ", "02", "3.75", "Nice -1.5 ", "12.30am", "1024", "Valenciennes vs Paris FC", "(+15)", "021/2 Goal", "01", "1.12", "Valenciennes +1.5 ", "02", "4.80", "Paris FC -1.5 ", "1.00am", "1357", "Roma vs Verona", "(+17)", "461/2 Goal", "01", "2.30", "Roma -1.5 ", "02", "1.50", "Verona +1.5 "]}

allmonths=set(['Jan','Feb','Mar','Apr','May','Jun',
                'Jul','Aug','Sep','Oct','Nov','Dec'])

def isEventDate(inp:str) -> bool:
    tempsplit = inp.split()
    for component in tempsplit:
        if component in allmonths:
            logging.info(f"Input String ({inp}) appears to be valid event date.")
            return True
    return False

def isEventTime(inp:str) -> bool:
    validtimeregexp = re.compile(r'\.[0-9]{2}[a,p]m')#allows .30am and .62pm #special case of 'Ham' (hamburg) or 'Jamshedpur' is declined.
    tempsplit = inp.split()
    for component in tempsplit:
        if validtimeregexp.search(component):
            logging.info(f"Input String ({inp}) appears to be valid event time.")
            return True
        # if "am" in component or "pm" in component:
        #     return True
    return False


# import re
# word = 'fubar'
# regexp = re.compile(r'ba[rzd]')
# if regexp.search(word):
#   print('matched')


def isEventId(inp:str) -> bool:
    try: 
        if eval(inp) == 2024 or 999> eval(inp) or eval(inp)>9999:
            return False
        logging.info(f"Input String ({inp}) appears to be valid event ID.")
        return True
    except:
        return False


eventdate=None
eventtime=None
eventid=None
AllEventStrings=[]
toJson = {}

select_bet = Select(bet_type_specific)
display_bet=driver.find_element(By.XPATH, "//button[@class='btn-block button button--orange btn btn-default']")
for bet_type in bet_type_list:
    select_bet.select_by_visible_text(bet_type)
    display_bet.click()
    time.sleep(3) #Wait for elements to load
    

    # Bet Type may not always be available; Skip to next bet type
    try:
        # Load More Button
        load_button = driver.find_element(By.CLASS_NAME,"event-list__load-all-events")
        load_button.click()
    except:
        continue

    events = driver.find_elements(By.XPATH,"//div[@class='event-list']//div[@class='event-list__group']")
    more_bets = driver.find_elements(By.CLASS_NAME, "show-all")
    for more in more_bets:
        more.click()
    for event in events:

        eventdate,eventtime,eventid,hometeam,awayteam,handicap=None,None,None,None,None,None
        
        eventdate,eventtime,eventid,hometeam,awayteam,handicap=event_filter(event.text)
        
        if eventtime is None:
            continue

        if eventdate not in toJson:
            toJson[eventdate] = {}
        if eventtime not in toJson[eventdate]:
            toJson[eventdate][eventtime]={}
        if eventid not in toJson[eventdate][eventtime]:
            toJson[eventdate][eventtime][eventid]={}


        toJson[eventdate][eventtime][eventid]['Home']=hometeam
        toJson[eventdate][eventtime][eventid]['Away']=awayteam
        
        if 'BetType' not in toJson[eventdate][eventtime][eventid]:
            toJson[eventdate][eventtime][eventid]['BetType']={}
        
        toJson[eventdate][eventtime][eventid]['BetType'][bet_type] = handicap
        




        # all_elem_in_events = event.text.split('\n')

        # eventdate=None
        # eventtime=None
        # eventid=None
        # EventString=""
    
        # for idx in range(len(all_elem_in_events)):
        #     elem = all_elem_in_events[idx]
        #     tempsplit = elem.split()
        #     for component in tempsplit:
        #         if isEventDate(component):
        #             eventdate = elem
        #             if eventdate not in toJson:
        #                 toJson[eventdate] = {}
        #         elif isEventTime(component):
        #             eventtime=component
        #             if eventtime not in toJson[eventdate]:
        #                 toJson[eventdate][eventtime]={}
        #         elif isEventId(component):
        #             eventid=component
        #             toJson[eventdate][eventtime][eventid]={}
                    
        #             if len(EventString)==0: # no previous string written; no need to write to memory
        #                 pass
        #             else:
        #                 AllEventStrings.append(EventString)
        #             EventString=f"{eventdate} {eventtime} {eventid} {bet_type} "
        #         elif "vs" in component:
        #             if component not in toJson[eventdate][eventtime][eventid]['Matchup']:
        #                 toJson[eventdate][eventtime][eventid]['Match:up'] += component
                
        #         else:
        #             EventString+=f" {component}"
        #             try:
        #                 toJson[eventdate][eventtime][eventid] = f"{component}"
        #             except:
        #                 pass        
                    
        # AllEventStrings.append(EventString)
        
        
        events_text = {bet_type : event.text.split('\n')}
        # events_text = {bet_type : event.json()}
        with open("sg_pools2.json",'a') as json_file:
            json_file.write(json.dumps(events_text))

driver.close()


KeyboardInterrupt: 

In [16]:
toJson

{'Thu, 14 Mar 2024': {'10.00pm': {'6533': {'Home': '6533\nFC Goa',
    'Away': 'Bengaluru',
    'BetType': {'1/2 Goal': '+4',
     '1X2': '+4',
     'Asian Handicap/HT Asian Handicap': '+4',
     'Halftime 1x2': '+4',
     'Halftime Total Goals': '+4',
     'Halftime-Fulltime': '+4',
     'Pick the Score': '+4',
     'Team to Score 1st Goal': '+4',
     'Total Goals': '+4',
     'Total Goals Odd/Even': '+4',
     'Total Goals Over/Under': '+4',
     'Will Both Teams Score': '+4'}}}}}

In [6]:
# Define patterns for different components

data = event.text


In [5]:
toJson

{'Thu, 14 Mar 2024': {'10.00pm': {'6533': 'Goa'}}}

In [ ]:
AllEventStrings

[' Fri, 8 2024 0682 Qarabag vs Leverkusen (+17) 021/2 Goal 01 1.77 Qarabag +1.5 02 1.87 Leverkusen -1.5 0675 Roma vs Brighton (+17) 461/2 Goal 01 3.15 Roma -1.5 02 1.28 Brighton +1.5 0690 Sparta Prague vs Liverpool (+18) 021/2 Goal 01 1.40 Sparta Prague +1.5 02 2.60 Liverpool -1.5 0683 Benfica vs Glasgow Rangers (+17) 461/2 Goal 01 2.45 Benfica -1.5 02 1.45 Glasgow Rangers +1.5 0605 Freiburg vs West (+17) 021/2 Goal 01 1.07 Freiburg +1.5 02 6.00 West -1.5 0609 Marseille vs Villarreal (+17) 461/2 Goal 01 2.80 Marseille -1.5 02 1.35 Villarreal +1.5 0685 Milan vs Slavia Prague (+17) 461/2 Goal 01 2.05 Milan -1.5 02 1.63 Slavia Prague +1.5',
 'Fri, 8 Mar 2024 4.45pm 4530 Western Sydney vs Western United (+17) 461/2 Goal 01 2.60 Western Sydney -1.5 02 1.40 Western United +1.5',
 'Fri, 8 Mar 2024 10.00pm 6527 vs Mumbai City (+4) 021/2 Goal 01 1.25 +1.5 02 3.35 Mumbai City -1.5',
 ' Sat, 9 2024',
 'Sat, 9 Mar 2024 1.30am 7573 Braunschweig vs Hansa Rostock (+15) 461/2 Goal 01 3.15 Braunschweig

In [ ]:
# f = open('sg_pools.txt','w')
# f.write(events_text)
# g=open('sg_pools_bettype.txt','w')
# g.write(bet_type_quantity1)

312

In [ ]:
s = "12.30am 1024"
k = s.split()
k

['12.30am', '1024']

In [ ]:
import logging
import datetime

logging.basicConfig(filename="sgpools_datacleaning.log",encoding='utf-8', level=logging.DEBUG)
current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
logging.info("Current Time: " + current_time)


testing_data = {"1/2 Goal": ["Sun, 21 Jan 2024", "12.30am", "1028", "Bordeaux vs Nice", "(+17)", "021/2 Goal", "01", "1.20", "Bordeaux +1.5 ", "02", "3.75", "Nice -1.5 ", "12.30am", "1024", "Valenciennes vs Paris FC", "(+15)", "021/2 Goal", "01", "1.12", "Valenciennes +1.5 ", "02", "4.80", "Paris FC -1.5 ", "1.00am", "1357", "Roma vs Verona", "(+17)", "461/2 Goal", "01", "2.30", "Roma -1.5 ", "02", "1.50", "Verona +1.5 "]}

allmonths=set(['Jan','Feb','Mar','Apr','May','Jun',
                'Jul','Aug','Sep','Oct','Nov','Dec'])

eventdate=None
eventtime=None
eventid=None


def isEventDate(inp:str) -> bool:
    tempsplit = inp.split()
    for component in tempsplit:
        if component in allmonths:
            logging.info(f"Input String ({inp}) appears to be valid event date.")
            return True
    return False

def isEventTime(inp:str) -> bool:
    tempsplit = inp.split()
    for component in tempsplit:
        if "am" in component or "pm" in component:
            logging.info(f"Input String ({inp}) appears to be valid event time.")
            return True
    return False


def isEventId(inp:str) -> bool:
    try: 
        if eval(inp) == 2024 or 999> eval(inp) or eval(inp)>9999:
            return False
        logging.info(f"Input String ({inp}) appears to be valid event ID.")
        return True
    except:
        return False
    
AllEventStrings = []
EventString = ""


# for idx in range(len(all_elem_in_events)):

# Simulating post scrape raw data
for idx in range(len(testing_data["1/2 Goal"])):
    elem = testing_data["1/2 Goal"][idx]
    # elem = all_elem_in_events[idx]
    tempsplit = elem.split()
    for component in tempsplit:
        if isEventDate(component):
            eventdate = elem
        elif isEventTime(component):
            eventtime=component
        elif isEventId(component):
            eventid=component
            if len(EventString)==0: # no previous string written; no need to write to memory
                pass
            else:
                AllEventStrings.append(EventString)
            EventString=f"{eventdate} {eventtime} {eventid}"
        else:
            EventString+=f" {component}"            
            
AllEventStrings.append(EventString)
    
    # if len(tempsplit) == 2:
    #         eventtime=tempsplit[0]
    #         eventid=tempsplit[1]
    #     elif len(tempsplit) == 1:
    #         eventtime=component 
    #     elif isEventId(component):
    #         eventid=component
        
    #     else:
    #         logging.debug(f"Unexpected Value: Component:{component} Context: {tempsplit}")
        


In [ ]:
AllEventStrings

[' Sun, 21 2024',
 'Sun, 21 Jan 2024 12.30am 1028 Bordeaux vs Nice (+17) 021/2 Goal 01 1.20 Bordeaux +1.5 02 3.75 Nice -1.5',
 'Sun, 21 Jan 2024 12.30am 1024 Valenciennes vs Paris FC (+15) 021/2 Goal 01 1.12 Valenciennes +1.5 02 4.80 Paris FC -1.5',
 'Sun, 21 Jan 2024 1.00am 1357 Roma vs Verona (+17) 461/2 Goal 01 2.30 Roma -1.5 02 1.50 Verona +1.5']

In [ ]:
with open('sgpools_datacleaning.log', 'w'):
    pass


In [ ]:
bet_type_list

['1/2 Goal',
 '1st Goal Scorer',
 '1X2',
 'Asian Handicap/HT Asian Handicap',
 'Halftime 1x2',
 'Halftime Total Goals',
 'Halftime-Fulltime',
 'Handicap 1X2',
 'Last Goal Scorer',
 'Pick the Score',
 'Team to Score 1st Goal',
 'Total Goals',
 'Total Goals Odd/Even',
 'Total Goals Over/Under',
 'Will Both Teams Score']

In [8]:
import re

testtext = 'Fri, 15 Mar 2024\n4.45pm\n4536\nNewcastle Jets vs Adelaide\n(+17)\n461/2 Goal\n01\n4.80\nNewcastle Jets -1.5 \n02\n1.12\nAdelaide +1.5 '
odds_pattern=r'0[1|2]\n(\d*\.\d*)\n[\w\s]* ([\+|\-][\d|\.]*)*'
odds_match = re.findall(odds_pattern,testtext,re.MULTILINE)

print(odds_match)

[('4.80', '-1.5'), ('1.12', '+1.5')]


In [7]:
odds_match.group(3)

IndexError: no such group

In [ ]:
import requests
url = "https://sg.1xbet.com/en/line/football"

url_response = requests.get(url)
url_response

<Response [200]>

In [ ]:
url_response.status_code==200

True